In [109]:
import pandas as pd
import requests

In [110]:
# Данные о визитах
visits_response = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
visits_data = visits_response.json()

In [111]:
# Данные о регистрациях
registrations_response = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
registrations_data = registrations_response.json()

In [112]:
visits_df = pd.DataFrame(visits_data)
registrations_df = pd.DataFrame(registrations_data)

In [113]:
# Удаление ботов
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]

# Преобразование datetime
visits_df['date'] = pd.to_datetime(visits_df['datetime']).dt.date

# Оставляем последний визит
visits_df = visits_df.sort_values('date').drop_duplicates('visit_id', keep='last')

In [114]:
# Группировка по дате и платформе
visits_grouped = visits_df.groupby(['date', 'platform']).size().reset_index(name='visits')

In [115]:
# Преобразование даты регистраций
registrations_df['date'] = pd.to_datetime(registrations_df['datetime']).dt.date

In [116]:
# Группировка по дате и платформе
registrations_grouped = registrations_df.groupby(['date', 'platform']).size().reset_index(name='registrations')

In [117]:
# Объединение датафреймов
df = pd.merge(visits_grouped, registrations_grouped, on=['date', 'platform'], how='outer').fillna(0)

In [118]:
# Расчет конверсии
df['conversion'] = (df['registrations'] / df['visits']) * 100

In [119]:
# Переименование столбца
df = df.rename(columns={'date': 'date_group'})

In [120]:
# Преобразование date_group в строковый формат
df['date_group'] = df['date_group'].astype(str)

In [122]:
# Сохранение в формате JSON
df.to_json('conversion.json', orient='columns')

In [123]:
print(df)

     date_group platform  visits  registrations  conversion
0    2023-03-01  android      75             61   81.333333
1    2023-03-01      ios      22             18   81.818182
2    2023-03-01      web     279              8    2.867384
3    2023-03-02  android      67             59   88.059701
4    2023-03-02      ios      31             24   77.419355
..          ...      ...     ...            ...         ...
547  2023-08-30      ios      66             40   60.606061
548  2023-08-30      web    1227             34    2.770986
549  2023-08-31  android      57             42   73.684211
550  2023-08-31      ios      50             36   72.000000
551  2023-08-31      web     721             46    6.380028

[552 rows x 5 columns]
